## 4. DOMAČA NALOGA

Odkrivanje enačb in uporaba predznanja
V tej nalogi je cilj najti (izmišljeno) enačbo ki opisuje prenos toplote v hladilnem stolpu. Podatki so
v datoteki DN4 1 podatki.csv. Hladilni stolp je naprava, ki se uporablja za vrnitev toplotnih
izgub v ozračje s hlajenjem vodnega pretoka na nižjo temperaturo. Za reševanje tega problema
lahko uporabiš poljubni algoritem za odkrivanje enačb. Predznanja ni nujno uporabljati, se pa
toplo priporoča, saj bo to močno zmanjšalo računsko zahtevnost naloge. Zanima nas povezava
med toplotnim tokom ter štirimi neodvisnimi spremenljivkami:

• spremenljivka na levi strani enačbe je Q, toplotni tok v vatih (W),

• Tw je temperatura vode v stopinjah Celzija,

• Ta je temperatura zraka v stopinjah Celzija,

• θ je kot med smerjo vetra in osjo stolpa v radianih, od 0 do π (0 pomeni, da sta veter in os
vzporedna),

• η je empirični izolacijski indeks, brez dimenzij.

Domensko predznanje

• Kot med smerjo vetra in osjo stolpa vpliva na toplotni tok, pri čemer se največ prenosa
toplote doseže, ko veter piha pravokotno na os stolpa.

• Razlika med temperaturo vode in zraka vpliva na toplotni tok, pri čemer višje razlike v
temperaturi prinašajo višji toplotni tok. To razmerje verjetno ni linearno.

• Toplotni tok verjetno ni odvisen od absolutnih vrednosti temperature vode ali zraka, ampak
samo od njune razlike.

• Empirični izolacijski indeks izhaja iz izmerjenih lastnosti tovarniškega procesa izdelave in
povzema termične lastnosti materiala vodnega stolpa. Nihče ga zares ne razume, vendar pa
vsak ve, da toplotni tok monotono pada z višanjem izolacijskega indeksa.

Poročilo naj vsebuje:
1. katero orodje si izbrala in zakaj,
2. natančen opis postopka reševanja, vključno z vsemi uporabljenimi nastavitvami orodja,
3. katero predznanje si uporabila in kako,
4. končno enačbo,
5. utemeljitev izbire enačbe ter komentar, koliko najdeni enačbi zaupaš in zakaj.

In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [140]:
podatki = pd.read_csv("DN4_1_podatki.csv")
print(podatki)

            Q         Tw         Ta     theta       eta
0    1.711929  53.005339  10.788147  0.321921  0.349083
1    0.067135  20.761340  12.225502  2.705824  0.821069
2    0.034759  39.546739  35.892183  1.873776  0.756082
3    0.417783  33.972185  21.385801  2.089589  0.516134
4    1.667614  50.092173  15.254621  0.328350  0.083327
..        ...        ...        ...       ...       ...
495  1.327393  54.081114  30.039247  1.153132  0.708757
496  0.053967  42.933431  39.799381  1.572222  0.022975
497  0.707523  47.145488  30.707064  2.093919  0.684035
498  0.062350  39.405105  32.647363  0.311168  0.053085
499  0.011049  47.549837  45.866743  2.258284  0.139831

[500 rows x 5 columns]


Poglejmo funkcije, ki smo jih uporabili pri vajah. S tem najprej ugotovimo kater od algoritmov bo za dane podatke najboljši.

In [141]:
def linearna_regresija(X, y, meja=1e-4):
    imena = X.columns
    beta = np.linalg.pinv(X.T.dot(X)).dot(X.T).dot(y)

    izraz = ""
    for i,b in enumerate(beta):
        if b > meja:
            if len(izraz) > 0:
                izraz += " + "
            izraz +=  f"{b:.3f}*{imena[i]}"
    return izraz


def ridge_regresija(X, y, lam=1, meja=1e-4):
    imena = X.columns
    beta = np.linalg.pinv(X.T.dot(X) + lam*np.identity(X.shape[1])).dot(X.T).dot(y)

    izraz = ""
    for i,b in enumerate(beta):
        if b > meja:
            if len(izraz) > 0:
                izraz += " + "
            izraz +=  f"{b:.3f}*{imena[i]}"
    return izraz


from scipy.optimize import minimize
def lasso_regresija(X, y, lam=1, meja=1e-4):
    imena = X.columns

    def f(beta):
        yhat = X.dot(beta)
        return np.sum((yhat-y)**2) + lam*np.sum(np.abs(beta))
    beta = minimize(f, np.random.random(X.shape[1]))["x"]
    
    izraz = ""
    for i,b in enumerate(beta):
        if b > meja:
            if len(izraz) > 0:
                izraz += " + "
            izraz +=  f"{b:.3f}*{imena[i]}"
    return izraz

Preizkusimo zgornje algoritme na naših podatkih.

Še prej si oglejmo domensko predznanje.

Domensko predznanje

• **Kot med smerjo vetra in osjo stolpa vpliva na toplotni tok, pri čemer se največ prenosa
toplote doseže, ko veter piha pravokotno na os stolpa.**

-- ko je $\theta$ enaka $\frac{\pi}{2}$ bo Q največji pri vseh ostalih konstanih spremenljivkah

-- hkrati pa bljižje kot bo $\theta$  $\frac{\pi}{2}$ večji bo Q 

• **Razlika med temperaturo vode in zraka vpliva na toplotni tok, pri čemer višje razlike v
temperaturi prinašajo višji toplotni tok. To razmerje verjetno ni linearno.**

-- (Tw - Ta) vecja kot je razika, večji je Q. Ne linearno.

Tw - temperatura vode

Ta - temperatura zraka 

• **Toplotni tok verjetno ni odvisen od absolutnih vrednosti temperature vode ali zraka, ampak
samo od njune razlike.**


• **Empirični izolacijski indeks izhaja iz izmerjenih lastnosti tovarniškega procesa izdelave in
povzema termične lastnosti materiala vodnega stolpa. Nihče ga zares ne razume, vendar pa
vsak ve, da toplotni tok monotono pada z višanjem izolacijskega indeksa.**

-- višji $\eta$ povzroči nižji toplotni tok

Glede na zgornje predpostavke oblikujemo naše podatke na tak način, da bo naša formula upoštevala razmerja med posameznimi spremenljivkami.

(1) Podatkom bomo dodali stolpec, ki bo transformacija stolpca s podatki $\theta$. Stolpec bo vseboval absolutno razliko med $\theta$ in vrednostjo $\frac{\pi}{2}$ = 1,571

(2) Velikost vpliva $\theta$ bi lahko upoštevali tudi tako, da podatkom dodamo stolpec $\sin(\theta)$, bližje kot bomo  $\frac{\pi}{2}$ višja bo vrednost sinusa pri tem kotu.

(3) Podatkom bomo dodali stolpec, ki bo transformacija stolpca s podatki o temperaturi vode in zraka. Stolpec bo vseboval njuni razliki - pri tem bomo upoštevali, da ta razlika ni nujno linearna. --> polinomski čeni (tako kot na vajah)

(4) Podatkom dodamo še zadnji stlpec, ki bo transofrmacija podatka o empiričnem indeksu, vemo namreč, da z večanjem indeksa zmanjšujemo toplotni tok. Za pravilno upoštevanje indeksa bomo zato ustvari stolpec, ki bo vseboval $\frac{1}{\eta^2}$ --> večji kot bo $\eta$ manjša bo vrednost podatka

In [142]:
# DODAMO STOLPCE Z USTREZNIMI TRANSFORMACIJAMI
from sklearn.preprocessing import PolynomialFeatures
import pandas as pd

podatki['razlika_med_theta_in_pi2'] = (np.pi/2 - podatki['theta']).abs()
podatki['sinus_theta'] = np.sin(podatki['theta'])

podatki['Tw_minus_Ta'] = (podatki['Tw'] - podatki['Ta'])

podatki['eta_2'] = 1/podatki['eta']**2

#print(podatki)

Uporabimo zgornje transformacije podatkov na naših algoritmih 

# 1. LINEARNA REGRESIJA

In [143]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(2)
X = poly.fit_transform(podatki.iloc[:, -3:])

#print(podatki.iloc[:, -3:])

imena_stolpcev = poly.get_feature_names_out()
X_df = pd.DataFrame(X, columns=imena_stolpcev)

In [144]:
linearna_regresija(X_df, podatki["Q"])

'0.002*Tw_minus_Ta^2'

# 2. RIDGE REGRESIJA

In [145]:

ridge_regresija(X_df, podatki["Q"], lam=0.5)

'0.002*Tw_minus_Ta^2'

# 3. LASSO REGRESIJA

In [146]:
lasso_regresija(X_df, podatki["Q"], lam=1000)

'0.381*sinus_theta + 0.142*eta_2 + 0.013*sinus_theta eta_2 + 0.095*Tw_minus_Ta^2'

# PREGLED NATANČNOSTI PREDLAGANIH ENAČB

Zgornji algoritmi so nam glede na podane podatke predlagani različne enačbe s katerimi bi lahko določili vrednost Q v odvisnosti od vrednosti različnih spremenljivk, ki nanj vplivajo. V nadaljevanju nas bo zanimalo kakšna je natančnost predlaganih enačb glede na dejansko vrednost Qja.

In [147]:
Q_lin = 0.002 * podatki['Tw_minus_Ta']**2
Q_ridge = 0.002 * podatki['Tw_minus_Ta']**2
Q_lasso = 1.747* podatki['sinus_theta'] + 0.143*podatki['eta_2'] + 1.145*podatki['sinus_theta']**2 + 0.013*podatki['sinus_theta'] * podatki['eta_2'] + 0.094*podatki['Tw_minus_Ta']**2
#print(podatki)

In [148]:
#RAZLIKE MED DEJANSKIM Q IN Q, KI NAM GA VRNEJO ZGORNJI ALGORITMI

from sklearn.metrics import mean_squared_error

#linearna regresija = ridge regresija 
mse_lin_reg = mean_squared_error(podatki['Q'], Q_lin)
mse_ridge_reg = mean_squared_error(podatki['Q'], Q_ridge)

mse_lasso_reg = mean_squared_error(podatki['Q'], Q_lasso)

print(mse_lin_reg)
print(mse_lasso_reg)

0.4319633566174793
2809881.2466726624


Manjša napaka med dejanskim Q in Q izračunanim z regresijo se pojavi pri linearni regresiji. Pri lasso regresiji je ta napaka precej visoka.

### *******************************************************************************************************************************

In [149]:
podatki

,Q,Tw,Ta,theta,eta,razlika_med_theta_in_pi2,sinus_theta,Tw_minus_Ta,eta_2
0,1.711929,53.005339,10.788147,0.321921,0.349083,1.248875,0.316390,42.217191,8.206192
1,0.067135,20.761340,12.225502,2.705824,0.821069,1.135028,0.422107,8.535838,1.483341
2,0.034759,39.546739,35.892183,1.873776,0.756082,0.302980,0.954452,3.654556,1.749294
3,0.417783,33.972185,21.385801,2.089589,0.516134,0.518793,0.868418,12.586385,3.753839
4,1.667614,50.092173,15.254621,0.328350,0.083327,1.242446,0.322482,34.837551,144.021782
...,...,...,...,...,...,...,...,...,...
495,1.327393,54.081114,30.039247,1.153132,0.708757,0.417664,0.914039,24.041866,1.990698
496,0.053967,42.933431,39.799381,1.572222,0.022975,0.001425,0.999999,3.134050,1894.479620
497,0.707523,47.145488,30.707064,2.093919,0.684035,0.523123,0.866263,16.438423,2.137193
498,0.062350,39.405105,32.647363,0.311168,0.053085,1.259628,0.306171,6.757743,354.865964


In [150]:
import ProGED as pg

np.random.seed(1)
ED1 = pg.EqDisco(data=podatki, 
                lhs_vars=["Q"],
                rhs_vars=["theta", "eta", "razlika_med_theta_in_pi2", "sinus_theta", "Tw_minus_Ta", "eta_2"],
                sample_size=100)

ED1.generate_models()
ED1.fit_models()

Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).


ModelBox: 65 models
-> [156.524057525843*eta*eta_2*theta + 2*eta - eta_2 - 505.414677766949 - 1.14246223072865/(-1.87299101791363*Tw_minus_Ta/sinus_theta + sinus_theta)], p = 4.2679544859209266e-39, error = 10112.013273016088, time = 1.1767299175262451
-> [sinus_theta], p = 0.024000000000000004, error = 1.0127283737714943, time = 0
-> [-sin(theta - 9.44969937611032)], p = 8.294400000000003e-06, error = 1.0125607996277983, time = 0.038263559341430664
-> [cos(theta)], p = 7.4649600000000045e-06, error = 1.4491875386616362, time = 0
-> [0.000418897970088092*eta*eta_2 - 0.136211300386157*razlika_med_theta_in_pi2 + sinus_theta + 0.149482920386556], p = 2.3084466527482714e-20, error = 1.0097729786841974, time = 0.173309326171875
-> [Tw_minus_Ta - 14.1566705084246 + 0.000400118502247798/(eta*theta)], p = 3.019898880000003e-12, error = 9.657679172843965, time = 0.12145662307739258
-> [5.6930943412995e-6*eta_2*theta], p = 2.5600000000000012e-05, error = 1.2707085657598123, time = 0.054237127304

In [151]:
ED1.get_results()

ModelBox: 1 models
-> [0.0771175316195712*Tw_minus_Ta - 0.459987439834972], p = 9.216000000000003e-05, error = 0.7081653733778198, time = 0.06689167022705078

Višja napaka kot prej, slabše. Dodamo v podatke kvadratr in poiskusimo tako

In [ ]:
import ProGED as pg

gramatika = "E -> E '+' F [0.1]| E '-' F [0.4]| F [0.5] \n"
gramatika = "F -> F '*' T [0.4]| F '/' T [0.2]| 'C' '*' T [0.4] \n"
gramatika += "T -> 'C' '*' V [0.5]| '('F')' [0.2] | 'sin' '('F')' [0.3] \n"
gramatika += "V  -> 'eta' [0.1]| 'theta' [0.3] | 'Tw' [0.3] | 'Ta' [0.3]"
grammar = pg.GeneratorGrammar(gramatika)

np.random.seed(1)
ED2 = pg.EqDisco(data=podatki, 
                lhs_vars=["Q"],
                rhs_vars=["theta", "eta", "Tw", "Ta"],
                sample_size=1000,
                generator = grammar)

ED2.generate_models()
ED2.fit_models()
ED2.get_results()

Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max constant).
Model skipped. More model parameters than allowed (check max con

ModelBox: 1 models
-> [0.460215746004746*Tw/Ta], p = 0.000144, error = 0.8869595230442364, time = 0.02766275405883789

In [88]:
podatki

,Q,Tw,Ta,theta,eta,razlika_med_theta_ in_ pi2,sinus_theta,Tw_-_Ta,1/eta^2
0,1.711929,53.005339,10.788147,0.321921,0.349083,1.248875,0.316390,42.217191,8.206192
1,0.067135,20.761340,12.225502,2.705824,0.821069,1.135028,0.422107,8.535838,1.483341
2,0.034759,39.546739,35.892183,1.873776,0.756082,0.302980,0.954452,3.654556,1.749294
3,0.417783,33.972185,21.385801,2.089589,0.516134,0.518793,0.868418,12.586385,3.753839
4,1.667614,50.092173,15.254621,0.328350,0.083327,1.242446,0.322482,34.837551,144.021782
...,...,...,...,...,...,...,...,...,...
495,1.327393,54.081114,30.039247,1.153132,0.708757,0.417664,0.914039,24.041866,1.990698
496,0.053967,42.933431,39.799381,1.572222,0.022975,0.001425,0.999999,3.134050,1894.479620
497,0.707523,47.145488,30.707064,2.093919,0.684035,0.523123,0.866263,16.438423,2.137193
498,0.062350,39.405105,32.647363,0.311168,0.053085,1.259628,0.306171,6.757743,354.865964


In [152]:
from pysr import PySRRegressor

model = PySRRegressor(
    niterations= 1000,
    binary_operators=['+','*','-','^','/'],
    unary_operators=['cos', 'sin', 'sqrt', 'square'],
    loss='loss(prediction, target) = (prediction - target)^2'
)

X = podatki.drop("Q", axis=1)
y = podatki["Q"]

model.fit(X,y)
model.latex()
print(model)

c:\Users\Uporabnik\AppData\Local\Programs\Python\Python311\Lib\site-packages\pysr\sr.py:1296: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(


Compiling Julia backend...


c:\Users\Uporabnik\AppData\Local\Programs\Python\Python311\Lib\site-packages\julia\juliainfo.py:93: UserWarning: julia warned:
The latest version of Julia in the `release` channel is 1.9.1+0.x64.w64.mingw32. You currently have `1.9.0+0.x64.w64.mingw32` installed. Run:

  juliaup update

to install Julia 1.9.1+0.x64.w64.mingw32 and update the `release` channel to that version.
  warnings.warn("{} warned:\n{}".format(julia, stderr))


PySRRegressor.equations_ = [
	    pick     score                                           equation  \
	0         0.000000                                        sinus_theta   
	1         0.292235                        (0.056379296 * Tw_minus_Ta)   
	2         0.285201                  square(Tw_minus_Ta * 0.045700118)   
	3         0.307830        ((Tw_minus_Ta * 0.087604776) * sinus_theta)   
	4         0.926354  (square(Tw_minus_Ta * -0.05715749) * sinus_theta)   
	5         0.400656  (square((Tw_minus_Ta * -0.071140714) + eta) * ...   
	6         1.074180  (square((Tw_minus_Ta * cos(eta)) * 0.065970734...   
	7   >>>>  0.178153  ((square(Tw_minus_Ta * 0.059228055) / (0.67033...   
	8         0.172884  ((square(Tw_minus_Ta * -0.051895067) * sinus_t...   
	9         0.070161  ((square(Tw_minus_Ta * -0.045416106) * sinus_t...   
	10        0.000007  (square(Tw_minus_Ta * -0.042200424) / (sqrt(0....   
	11        0.001132  ((square(Tw_minus_Ta * -0.04543934) * (sinus_t...   
	12      

Zmagovalni model

In [154]:
Q_pysr = ((podatki['Tw_minus_Ta'] * 0.059228055)**2 / (0.670336 + podatki['eta'])) * podatki['sinus_theta']
mse_pysr = mean_squared_error(podatki['Q'], Q_pysr)
mse_pysr

0.0160474864686204